Id in Image corresponds to the annotation "id"
at the bottom the categories are listed having "ids" for each category

## Global Variables

In [1]:
dataset_name = "MIT-Dataset" # CHANGE

dataset_path = "../MIT-Dataset/images" # CHANGE
dataset_labels = "../MIT-Dataset/annotations/"

# Source Dataset Paths
id_path = "../Dataset/id.txt"
prices_path = "Dataset/prices.txt"
destination = "Dataset/MIT-Dataset"
train = "D2S_training.json"

## Extract next ID and Lable count

In [2]:
with open(id_path, 'r') as f:
    lines = f.readlines()
    id_count = int(lines[0].split()[1]) + 1
    label_count = int(lines[1].split()[1]) + 1

print("Id count: ", id_count)
print("Label count: ", label_count)

Id count:  27549
Label count:  39


### taking a look at categories:

In [5]:
import json
import os
from PIL import Image, ImageDraw, ImageFont

def extract_unique_categories(annotation_path):
    with open(annotation_path, 'r') as f:
        coco_data = json.load(f)
    
    # Extrahiere Kategorien und Superkategorien
    categories = coco_data.get("categories", [])
    category_labels = {category["name"]: category["id"] for category in categories}
    supercategory_labels = {category["supercategory"]: category["id"] for category in categories}
    
    return category_labels, supercategory_labels, coco_data

def find_image_for_supercategory(coco_data, supercategory_name, dataset_path):
    """
    Finde ein Bild, das eine Kategorie aus der angegebenen Superkategorie enthält.
    """
    # Alle Kategorien, die zur Superkategorie gehören
    categories = [
        cat["id"] for cat in coco_data["categories"] 
        if cat["supercategory"] == supercategory_name
    ]
    
    # Finde die erste Annotation, die eine dieser Kategorien hat
    for annotation in coco_data["annotations"]:
        if annotation["category_id"] in categories:
            # Finde das zugehörige Bild
            image_id = annotation["image_id"]
            for image in coco_data["images"]:
                if image["id"] == image_id:
                    return os.path.join(dataset_path, image["file_name"]), annotation["category_id"]

    return None, None

def display_image_with_supercategory(image_path, supercategory_name):
    """
    Öffne ein Bild und schreibe die Superkategorie darüber.
    """
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    # Passe die Schriftart an (oder verwende den Standard)
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()  # Verwende Standardschrift, wenn die angegebene Schrift fehlt
    
    # Schreibe die Superkategorie oben ins Bild
    text = f"Supercategory: {supercategory_name}"
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
    
    # Zeichne einen Hintergrund für den Text
    draw.rectangle(((0, 0), (text_width + 10, text_height + 10)), fill="black")
    draw.text((5, 5), text, fill="white", font=font)
    
    # Bild anzeigen
    image.show()

categories = set()
supercategories = set()

for annons in os.listdir(dataset_labels):
    annotation_path = os.path.join(dataset_labels, annons)
    
    categories_temp, supercategories_temp, coco_data = extract_unique_categories(annotation_path)
    categories.update(categories_temp)  # Setze die neuen Kategorien ohne Duplikate
    supercategories.update(supercategories_temp)  # Setze die neuen Superkategorien ohne Duplikate

# Konvertiere zurück in Listen, falls notwendig
categories = list(categories)
supercategories = list(supercategories)

print(categories)
print(supercategories)

#Für jede Superkategorie ein Bild anzeigen
for supercategory_name in supercategories:
    image_path, category_id = find_image_for_supercategory(coco_data, supercategory_name, dataset_path)
    if image_path:
        print(f"Zeige Bild für Superkategorie: {supercategory_name} (Kategorie-ID: {category_id})")
        display_image_with_supercategory(image_path, supercategory_name)
    else:
        print(f"Kein Bild gefunden für Superkategorie: {supercategory_name}")


['gepa_bio_und_fair_pfefferminztee', 'roma_vine_tomatoes', 'adelholzener_gourmet_mineralwasser_02', 'franken_tafelreiniger', 'douwe_egberts_professional_ground_coffee', 'grapes_green_sugraone_seedless', 'kiwi', 'salad_iceberg', 'corny_nussvoll_single', 'avocado', 'zucchini', 'ethiquable_gruener_tee_ceylon', 'koelln_muesli_fruechte', 'gepa_bio_und_fair_fencheltee', 'cucumber', 'augustiner_weissbier_05', 'apple_granny_smith', 'corny_schoko_banane', 'gepa_bio_und_fair_rooibostee', 'apple_braeburn_bundle', 'vine_tomatoes', 'clementine', 'dr_oetker_vitalis_knuspermuesli_klassisch', 'oranges', 'augustiner_lagerbraeu_hell_05', 'gepa_bio_und_fair_kraeuterteemischung', 'apple_red_boskoop', 'pasta_reggia_fusilli', 'caona_cocoa', 'tegernseer_hell_03', 'grapes_sweet_celebration_seedless', 'cafe_wunderbar_espresso', 'corn_salad', 'corny_nussvoll', 'lettuce', 'cocoba_cocoa', 'adelholzener_alpenquelle_naturell_075', 'pasta_reggia_elicoidali', 'adelholzener_classic_naturell_02', 'banana_bundle', 'gepa

look at subcategories of unclear super categories

In [6]:
import json
import os
from PIL import Image, ImageDraw, ImageFont

def find_unwanted_subcategories(annotation_path, wanted_super_categories):
    with open(annotation_path, 'r') as f:
        coco_data = json.load(f)
    
    categories = coco_data.get("categories", [])
    unwanted_subcategories = {}
    
    for category in categories:
        supercategory_name = category["supercategory"]
        category_name = category["name"]
        category_id = category["id"]
        
        if supercategory_name not in wanted_super_categories:
            if supercategory_name not in unwanted_subcategories:
                unwanted_subcategories[supercategory_name] = []
            unwanted_subcategories[supercategory_name].append({"name": category_name, "id": category_id})
    
    return unwanted_subcategories

def display_images_with_subcategories(annotation_path, dataset_path, unwanted_subcategories):
    with open(annotation_path, 'r') as f:
        coco_data = json.load(f)
    
    annotations = coco_data.get("annotations", [])
    images = coco_data.get("images", [])
    
    # Map Image IDs to file names
    image_id_to_filename = {img["id"]: img["file_name"] for img in images}
    
    for supercategory, subcategories in unwanted_subcategories.items():
        for subcategory in subcategories:
            subcategory_name = subcategory["name"]
            subcategory_id = subcategory["id"]
            
            # Finde eine Annotation für diese Subkategorie
            for annotation in annotations:
                if annotation["category_id"] == subcategory_id:
                    image_id = annotation["image_id"]
                    image_file = image_id_to_filename.get(image_id)
                    
                    if image_file:
                        image_path = os.path.join(dataset_path, image_file)
                        
                        # Öffne das Bild
                        image = Image.open(image_path)
                        draw = ImageDraw.Draw(image)
                        
                        # Schrift für die Beschriftung (falls vorhanden, sonst Standard)
                        font = ImageFont.load_default()
                        
                        # Schreibe die Subkategorie auf das Bild
                        text = f"Subcategory: {subcategory_name}"
                        text_size = draw.textbbox((0, 0), text, font=font)
                        draw.rectangle([text_size[0], text_size[1], text_size[2] + 10, text_size[3] + 10], fill="black")
                        draw.text((5, 5), text, fill="white", font=font)
                        
                        # Zeige das Bild an
                        image.show()
                        break  # Nur ein Bild pro Subkategorie anzeigen

# Beispiel-Aufruf
wanted_super_categories = ["tea", "cereals", "packaged cereal bars", "salad", "pasta", "canned soft drink",
                           "board eraser", "pears", "kiwis", "bananas", "avocados", "orange citrus", "apples",
                           "zucchinis", "cucumbers", "carrots", "salad", "apple spritzer", "coca cola", "brown glass", "ink cartridge"]

merge_subcategories = {"water_small": ["adelholzener_gourmet_mineralwasser_02", "adelholzener_classic_naturell_02"], "water_big" : ["adelholzener_alpenquelle_naturell_075", "adelholzener_alpenquelle_classic_075"], "tomatos": ["vine_tomatoes","roma_vine_tomatoes"], 
                       "apple bundle": ["apple_braeburn_bundle "], "orange bundle": ["oranges", "clementine"], "grapes bundle" :["grapes_green_sugraone_seedless", "grapes_sweet_celebration_seedless"],
                       "salad": ["corn_salad", "rocket", "salad_iceberg"], "kaffee": ["gepa_bio_caffe_crema", "douwe_egberts_professional_ground_coffee", "cafe_wunderbar_espresso"], "lose Riegel": ["corny_schoko_banane_single", "corny_nussvoll_single"], "cacao": ["caona_cocoa", "cocoba_cocoa"]}

#TODO:: look at aple spritzer...

unwanted_subcategories = find_unwanted_subcategories(annotation_path, wanted_super_categories)
display_images_with_subcategories(annotation_path, dataset_path, unwanted_subcategories)

# Ausgabe der unerwünschten Subkategorien
for supercategory, subcategories in unwanted_subcategories.items():
    print(f"Supercategory: {supercategory}")
    for subcategory in subcategories:
        print(f"  - Subcategory: {subcategory['name']} (ID: {subcategory['id']})")


Supercategory: textured packaging
  - Subcategory: caona_cocoa (ID: 19)
  - Subcategory: cafe_wunderbar_espresso (ID: 21)
  - Subcategory: gepa_italienischer_bio_espresso (ID: 24)
  - Subcategory: douwe_egberts_professional_ground_coffee (ID: 22)
  - Subcategory: corny_nussvoll_single (ID: 13)
  - Subcategory: corny_schoko_banane_single (ID: 15)
  - Subcategory: gepa_bio_caffe_crema (ID: 23)
  - Subcategory: cocoba_cocoa (ID: 20)
Supercategory: plastic tray
  - Subcategory: grapes_green_sugraone_seedless (ID: 34)
  - Subcategory: corn_salad (ID: 54)
  - Subcategory: rocket (ID: 58)
  - Subcategory: grapes_sweet_celebration_seedless (ID: 35)
Supercategory: nets
  - Subcategory: oranges (ID: 38)
  - Subcategory: clementine (ID: 32)
Supercategory: foil
  - Subcategory: salad_iceberg (ID: 59)
Supercategory: cardboard tray
  - Subcategory: vine_tomatoes (ID: 56)
  - Subcategory: apple_braeburn_bundle (ID: 25)
  - Subcategory: roma_vine_tomatoes (ID: 57)
Supercategory: water
  - Subcategory:

In [7]:
import json

def map_subcategories(annotation_path, wanted_super_categories, merge_subcategories):
    with open(annotation_path, 'r') as f:
        coco_data = json.load(f)

    categories = coco_data.get("categories", [])
    subcategory_to_supercategory = {}
    not_covered_subcategories = []

    for category in categories:
        category_name = category["name"]
        supercategory_name = category["supercategory"]
        
        # Prüfen, ob die Subkategorie in wanted_super_categories enthalten ist
        if supercategory_name in wanted_super_categories:
            subcategory_to_supercategory[category_name] = supercategory_name
        # Prüfen, ob die Subkategorie in merge_subcategories enthalten ist
        elif any(category_name in subcategories for subcategories in merge_subcategories.values()):
            # Finde die passende Superkategorie in merge_subcategories
            for merged_supercategory, subcategories in merge_subcategories.items():
                if category_name in subcategories:
                    subcategory_to_supercategory[category_name] = merged_supercategory
                    break
        else:
            # Subkategorie ist nicht abgedeckt
            not_covered_subcategories.append(category_name)

    return subcategory_to_supercategory, not_covered_subcategories

# Beispiel-Aufruf
wanted_super_categories = [
    "tea", "cereals", "packaged cereal bars", "salad", "pasta", "canned soft drink",
    "board eraser", "pears", "kiwis", "bananas", "avocados", "orange citrus", "apples",
    "zucchinis", "cucumbers", "carrots", "salad", "apple spritzer", "coca cola", "brown glass", "ink cartridge"
]
merge_subcategories = {
    "water_small": ["adelholzener_gourmet_mineralwasser_02", "adelholzener_classic_naturell_02"],
    "water_big": ["adelholzener_alpenquelle_naturell_075", "adelholzener_alpenquelle_classic_075"],
    "tomatos": ["vine_tomatoes", "roma_vine_tomatoes"],
    "apple bundle": ["apple_braeburn_bundle"],
    "orange bundle": ["oranges", "clementine"],
    "grapes bundle": ["grapes_green_sugraone_seedless", "grapes_sweet_celebration_seedless"],
    "salad": ["corn_salad", "rocket", "salad_iceberg"],
    "kaffee": ["gepa_bio_caffe_crema", "douwe_egberts_professional_ground_coffee", "cafe_wunderbar_espresso", "gepa_italienischer_bio_espresso"],
    "lose Riegel": ["corny_schoko_banane_single", "corny_nussvoll_single"],
    "cacao": ["caona_cocoa", "cocoba_cocoa"]
}


subcategory_to_supercategory, not_covered_subcategories = map_subcategories(
    annotation_path, wanted_super_categories, merge_subcategories
)

# Ausgabe: Subkategorie -> Superkategorie
print("Subcategory to Supercategory Mapping:")
for subcategory, supercategory in subcategory_to_supercategory.items():
    print(f"{subcategory} -> {supercategory}")

# Ausgabe: Nicht abgedeckte Subkategorien
print("\nNot Covered Subcategories:")
for subcategory in not_covered_subcategories:
    print(subcategory)


Subcategory to Supercategory Mapping:
gepa_bio_und_fair_fencheltee -> tea
gepa_bio_und_fair_kraeuterteemischung -> tea
gepa_bio_und_fair_kamillentee -> tea
ethiquable_gruener_tee_ceylon -> tea
gepa_bio_und_fair_pfefferminztee -> tea
gepa_bio_und_fair_rooibostee -> tea
kilimanjaro_tea_earl_grey -> tea
koelln_muesli_fruechte -> cereals
koelln_muesli_schoko -> cereals
dr_oetker_vitalis_knuspermuesli_klassisch -> cereals
corny_nussvoll -> packaged cereal bars
corny_schoko_banane -> packaged cereal bars
pelikan_tintenpatrone_canon -> ink cartridge
lettuce -> salad
carrot -> carrots
cucumber -> cucumbers
zucchini -> zucchinis
apple_golden_delicious -> apples
apple_granny_smith -> apples
apple_red_boskoop -> apples
orange_single -> orange citrus
clementine_single -> orange citrus
avocado -> avocados
banana_bundle -> bananas
banana_single -> bananas
kiwi -> kiwis
pear -> pears
franken_tafelreiniger -> board eraser
suntory_gokuri_lemonade -> canned soft drink
pasta_reggia_elicoidali -> pasta
pa

In [41]:
wanted_super_categories = [
    "tea", "cereals", "packaged cereal bars", "salad", "pasta", "canned soft drink",
    "board eraser", "pears", "kiwis", "bananas", "avocados", "orange citrus", "apples",
    "zucchinis", "cucumbers", "carrots", "apple spritzer", "coca cola", "brown glass", "ink cartridge"
]

merge_subcategories = {
    "water_small": ["adelholzener_gourmet_mineralwasser_02", "adelholzener_classic_naturell_02"],
    "water_big": ["adelholzener_alpenquelle_naturell_075", "adelholzener_alpenquelle_classic_075"],
    "tomatos": ["vine_tomatoes", "roma_vine_tomatoes"],
    "apple bundle": ["apple_braeburn_bundle"],
    "orange bundle": ["oranges", "clementine"],
    "grapes bundle": ["grapes_green_sugraone_seedless", "grapes_sweet_celebration_seedless"],
    "salad": ["corn_salad", "rocket", "salad_iceberg"],
    "coffee": ["gepa_bio_caffe_crema", "douwe_egberts_professional_ground_coffee", "cafe_wunderbar_espresso", "gepa_italienischer_bio_espresso"],
    "lose Riegel": ["corny_schoko_banane_single", "corny_nussvoll_single"],
    "cacao": ["caona_cocoa", "cocoba_cocoa"]
}

existing = {"tea": 26, "pasta": 9, "bananas": 4, "avocados": 3, "apples": 1, "coffee": 13}

labels = []
for label in wanted_super_categories:
    labels.append(label)
for key in merge_subcategories:
    labels.append(key)
0.

label_count = 39
label_to_price = {}
num_to_label = {}
i = 0
for label in labels:
    if not label in existing:
        new_label = input(f"Enter the new label for {label} (Skip): ")
        price = float(input(f"Enter the price for {label}: "))
        label_to_price[label_count] = [new_label,price]
        num_to_label[i] = label_count
        label_count += 1
        print("Added label and price:", new_label, price)
        i += 1

print(label_to_price)

label_to_price_with_existing = {**label_to_price, **existing}
print(label_to_price_with_existing)
price_id_dict = {39: ['cereals', 2.49], 40: ['packaged cereal bars', 1.99], 41: ['salad', 2.29], 42: ['canned soft drink', 0.99], 
        43: ['board eraser', 4.99], 44 : ['pears', 0.67], 45: ['kiwis', 0.29], 46: ['orange citrus', 0.29], 47: ['zucchinis', 1.11],
        48: ['cucumbers', 1.11], 49: ['carrots', 0.19], 50: ['apple spritzer', 1.19], 51: ['coca cola', 1.10], 52: ['brown glass', 1.29],
        53: ['ink cartridge', 4.99], 54: ['water_small', 0.79], 55: ['water_big', 1.19], 56: ['tomatos', 1.49], 57: ['apple bundle', 1.19],
        58: ['orange bundle', 1.29], 59: ['grapes bundle', 1.49], 60: ['lose Riegel', 0.69], 61: ['cacao', 4.99],
        26: ['tea', 2.35], 9: ['pasta', 2.69], 3 : ['avocados', 1.29],
        4: ['bananas', 0.46], 1: ['apples',  0.6], 13: ['coffee', 7.99]}

Added label and price:  2.49
Added label and price:  1.99
Added label and price:  2.29
Added label and price:  0.99
Added label and price:  4.0
Added label and price:  0.67
Added label and price:  0.79
Added label and price: banana 0.46
Added label and price:  0.8
Added label and price:  1.11
Added label and price:  1.11
Added label and price:  0.15
Added label and price:  1.49
Added label and price:  1.19
Added label and price:  1.19
Added label and price:  1.7
Added label and price:  9.99
Added label and price:  0.8
Added label and price:  1.29
Added label and price:  2.5
Added label and price:  1.2
Added label and price:  1.2
Added label and price:  1.99
Added label and price:  1.49
Added label and price:  0.69
Added label and price:  5.99
{39: ['', 2.49], 40: ['', 1.99], 41: ['', 2.29], 42: ['', 0.99], 43: ['', 4.0], 44: ['', 0.67], 45: ['', 0.79], 46: ['banana', 0.46], 47: ['', 0.8], 48: ['', 1.11], 49: ['', 1.11], 50: ['', 0.15], 51: ['', 1.49], 52: ['', 1.19], 53: ['', 1.19], 54

In [ ]:
wanted_super_categories = [
    "tea", "cereals", "packaged cereal bars", "salad", "pasta", "canned soft drink",
    "board eraser", "pears", "kiwis", "bananas", "avocados", "orange citrus", "apples",
    "zucchinis", "cucumbers", "carrots", "apple spritzer", "coca cola", "brown glass", "ink cartridge"
]

merge_subcategories = {
    "water_small": ["adelholzener_gourmet_mineralwasser_02", "adelholzener_classic_naturell_02"],
    "water_big": ["adelholzener_alpenquelle_naturell_075", "adelholzener_alpenquelle_classic_075"],
    "tomatos": ["vine_tomatoes", "roma_vine_tomatoes"],
    "apple bundle": ["apple_braeburn_bundle"],
    "orange bundle": ["oranges", "clementine"],
    "grapes bundle": ["grapes_green_sugraone_seedless", "grapes_sweet_celebration_seedless"],
    "salad": ["corn_salad", "rocket", "salad_iceberg"],
    "coffee": ["gepa_bio_caffe_crema", "douwe_egberts_professional_ground_coffee", "cafe_wunderbar_espresso", "gepa_italienischer_bio_espresso"],
    "lose Riegel": ["corny_schoko_banane_single", "corny_nussvoll_single"],
    "cacao": ["caona_cocoa", "cocoba_cocoa"]
}

price_id_dict = {39: ['cereals', 2.49], 40: ['packaged cereal bars', 1.99], 41: ['salad', 2.29], 42: ['canned soft drink', 0.99], 
        43: ['board eraser', 4.99], 44 : ['pears', 0.67], 45: ['kiwis', 0.29], 46: ['orange citrus', 0.29], 47: ['zucchinis', 1.11],
        48: ['cucumbers', 1.11], 49: ['carrots', 0.19], 50: ['apple spritzer', 1.19], 51: ['coca cola', 1.10], 52: ['brown glass', 1.29],
        53: ['ink cartridge', 4.99], 54: ['water_small', 0.79], 55: ['water_big', 1.19], 56: ['tomatos', 1.49], 57: ['apple bundle', 1.19],
        58: ['orange bundle', 1.29], 59: ['grapes bundle', 1.49], 60: ['lose Riegel', 0.69], 61: ['cacao', 4.99],
        26: ['tea', 2.35], 9: ['pasta', 2.69], 3 : ['avocados', 1.29],
        4: ['bananas', 0.46], 1: ['apples',  0.6], 13: ['coffee', 7.99]}


import os
import json
import shutil

# Pfade
dataset_path = "../MIT-Dataset/images/"
destination = "../Dataset/MIT-Dataset/"
dataset_labels = "../MIT-Dataset/annotations/"
id_path = "../Dataset/id.txt"

desired_files = ["D2S_test_info.json", "D2S_augmented.json", "D2S_training.json", "D2S_validation.json"]

def process_annotations():
    image_id_counter = 4044

    # Hole alle Bilder im Dataset-Label-Verzeichnis
    for annotation_file in desired_files:
        anotation_path = os.path.join(dataset_labels, annotation_file)
        with open(anotation_path, 'r') as f:
                coco_data = json.load(f)

        for image_coco in coco_data.get("images", []):
            imgage_id = image_coco["id"]

            image_annotations = []

            # Durchsuche die Annotations und finde die relevanten für das aktuelle Bild
            for annotation in coco_data.get("annotations", []):
                if annotation["image_id"] == int(imgage_id):  # Extrahiere die Bild-ID
                    image_annotations.append(annotation)

            # Wenn Annotationen vorhanden sind, bearbeite das Bild
            if image_annotations:
                objects_count = {}
                total_price = 0.0

                # Gehe durch alle gefundenen Annotationen
                for annotation in image_annotations:
                    category_id = annotation["category_id"]
                    category_name = None
                    supercategory_name = None

                    # Finde den entsprechenden Category-Namen und Supercategory
                    for category in coco_data["categories"]:
                        if category["id"] == category_id:
                            supercategory_name = category["supercategory"]
                            category_name = category["name"]
                            break
                    
                    # Bestimme den Namen basierend auf den supercategories und merge_subcategories
                    if supercategory_name in wanted_super_categories:
                        object_name = supercategory_name
                    else:
                        object_name = None
                        for merge_key, subcategories in merge_subcategories.items():
                            if category_name in subcategories:
                                object_name = merge_key
                                break
                    
                    # Falls ein passender Name gefunden wurde
                    if object_name:
                        for key, value in price_id_dict.items():
                            if value[0] == object_name:
                                object_id = key
                                break
                        if object_id not in objects_count:
                            objects_count[object_id] = 0
                        objects_count[object_id] += 1

                        # Berechne den Gesamtpreis
                        if category_id in price_id_dict:
                            total_price += price_id_dict[category_id][1]
                    else:
                        print("Found new object super:", supercategory_name, "sub", category_name)

                # Formatierte Bild-ID mit führenden Nullen
                formatted_image_id = f"image_{str(image_id_counter)}"
                new_image_filename = f"{formatted_image_id}.jpg"
                image_filename = f"D2S_{str(imgage_id).zfill(6)}.jpg"

                # Kopiere das Bild
                image_path = os.path.join(dataset_path, image_filename)
                new_image_path = os.path.join(destination, new_image_filename)
                shutil.copy(image_path, new_image_path)

                # Erstelle die TXT-Datei mit den Objecten und Preisen
                txt_filename = f"{formatted_image_id}.txt"
                txt_file_path = os.path.join(destination, txt_filename)
                with open(txt_file_path, 'w') as f:
                    f.write(f"Objects: {str(objects_count)}\n")
                    f.write(f"Total Price: {total_price:.2f}")

                image_id_counter += 1
                print(f"Bild {image_filename} verarbeitet und nach {new_image_filename} verschoben.")
    with open(id_path, "r") as f:
        lines = f.readlines()

    if lines:
        lines[0] = "current_id: " + str(image_id_counter) + "\n"
    else:
        lines.append("current_id: " + str(image_id_counter) + "\n")

    with open(id_path, "w") as f:
        f.writelines(lines)

    

# Beispiel-Aufruf
process_annotations()

Bild D2S_99000000.jpg verarbeitet und nach image_3174.jpg verschoben.
Bild D2S_99000001.jpg verarbeitet und nach image_3175.jpg verschoben.
Bild D2S_99000002.jpg verarbeitet und nach image_3176.jpg verschoben.
Bild D2S_99000003.jpg verarbeitet und nach image_3177.jpg verschoben.
Bild D2S_99000004.jpg verarbeitet und nach image_3178.jpg verschoben.
Bild D2S_99000005.jpg verarbeitet und nach image_3179.jpg verschoben.
Bild D2S_99000006.jpg verarbeitet und nach image_3180.jpg verschoben.
Bild D2S_99000007.jpg verarbeitet und nach image_3181.jpg verschoben.
Bild D2S_99000008.jpg verarbeitet und nach image_3182.jpg verschoben.
Bild D2S_99000009.jpg verarbeitet und nach image_3183.jpg verschoben.
Bild D2S_99000010.jpg verarbeitet und nach image_3184.jpg verschoben.
Bild D2S_99000011.jpg verarbeitet und nach image_3185.jpg verschoben.
Bild D2S_99000012.jpg verarbeitet und nach image_3186.jpg verschoben.
Bild D2S_99000013.jpg verarbeitet und nach image_3187.jpg verschoben.
Bild D2S_99000014.jp

In [44]:
import json

def count_images_with_annotations(annotation_path):
    with open(annotation_path, 'r') as f:
        coco_data = json.load(f)
    
    # Extrahiere alle Annotationen und Bilder
    annotations = coco_data.get("annotations", [])
    images = coco_data.get("images", [])
    
    # Setze die Bild-IDs aus den Annotationen
    annotated_image_ids = {annotation["image_id"] for annotation in annotations}
    
    # Zähle die Bilder, die eine Annotation haben
    count = sum(1 for image in images if image["id"] in annotated_image_ids)
    
    return count

# Beispiel-Aufruf
dataset_labels = "../MIT-Dataset/annotations/"
for subfolder in os.listdir(dataset_labels):
    path_to_labels = os.path.join(dataset_labels, subfolder)
    num_images_with_annotations = count_images_with_annotations(path_to_labels)
    print(f"Anzahl der Bilder mit Annotationen in {path_to_labels}: {num_images_with_annotations}")


Anzahl der Bilder mit Annotationen in ../MIT-Dataset/annotations/D2S_augmented.json: 10000
Anzahl der Bilder mit Annotationen in ../MIT-Dataset/annotations/D2S_test_info.json: 0
Anzahl der Bilder mit Annotationen in ../MIT-Dataset/annotations/D2S_test_info_clutter.json: 0
Anzahl der Bilder mit Annotationen in ../MIT-Dataset/annotations/D2S_test_info_occlusion.json: 0
Anzahl der Bilder mit Annotationen in ../MIT-Dataset/annotations/D2S_test_info_random_background.json: 0
Anzahl der Bilder mit Annotationen in ../MIT-Dataset/annotations/D2S_test_info_random_background_wo_clutter.json: 0
Anzahl der Bilder mit Annotationen in ../MIT-Dataset/annotations/D2S_test_info_wo_occlusion.json: 0
Anzahl der Bilder mit Annotationen in ../MIT-Dataset/annotations/D2S_training.json: 4380
Anzahl der Bilder mit Annotationen in ../MIT-Dataset/annotations/D2S_training_light0.json: 1460
Anzahl der Bilder mit Annotationen in ../MIT-Dataset/annotations/D2S_training_rot0.json: 438
Anzahl der Bilder mit Annotatio

In [10]:
import os
import ast

# Dictionary
price_id_dict = {
    39: ['cereals', 2.49], 40: ['packaged cereal bars', 1.99], 41: ['salad', 2.29],
    42: ['canned soft drink', 0.99], 43: ['board eraser', 4.99], 44: ['pears', 0.67],
    45: ['kiwis', 0.29], 46: ['orange citrus', 0.29], 47: ['zucchinis', 1.11],
    48: ['cucumbers', 1.11], 49: ['carrots', 0.19], 50: ['apple spritzer', 1.19],
    51: ['coca cola', 1.10], 52: ['brown glass', 1.29], 53: ['ink cartridge', 4.99],
    54: ['water_small', 0.79], 55: ['water_big', 1.19], 56: ['tomatos', 1.49],
    57: ['apple bundle', 1.19], 58: ['orange bundle', 1.29], 59: ['grapes bundle', 1.49],
    60: ['lose Riegel', 0.69], 61: ['cacao', 4.99], 26: ['tea', 2.35],
    9: ['pasta', 2.69], 3: ['avocados', 1.29], 4: ['bananas', 0.46],
    1: ['apples', 0.6], 13: ['coffee', 7.99]
}

# File path
path = "../Dataset/prices.txt"
name = "MITec: "

# Schreiben in die Datei
if os.path.exists(path):
    with open(path, "a") as f:  # "a" zum Anhängen
        f.write(f"{name}{price_id_dict}\n")
else:
    with open(path, "w") as f:  # "w" zum Erstellen und Schreiben
        f.write(f"{name}{price_id_dict}\n")
